# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.webservice import Webservice
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model

print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# Name of the experiment
experiment_name = 'titanic-automl-experiment'
experiment=Experiment(ws, experiment_name)

# Get the data of Kaggle Titanic Dataset
key = "titanic-modified"
description_text = "Kaggle Titanic Challenge dataset with some changes made by myself"
found = False

if key in ws.datasets.keys(): 
    found = True
    dataset = ws.datasets[key] 

if not found:
    # Create AML Dataset and register it into Workspace
    example_data = 'https://raw.githubusercontent.com/clasimoes/nd00333-capstone/master/titanic_data/full_capstone.csv'
    dataset = Dataset.Tabular.from_delimited_files(example_data)
    #Register Dataset in Workspace
    dataset = dataset.register(workspace=ws,
                               name=key,
                               description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Q,S,male
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.066409,0.523008,0.381594,32.204208,0.086420,0.725028,0.647587
std,257.353842,0.486592,0.836071,13.244532,1.102743,0.806057,49.693429,0.281141,0.446751,0.477990
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400,0.000000,0.000000,0.000000
50%,446.000000,0.000000,3.000000,26.000000,0.000000,0.000000,14.454200,0.000000,1.000000,1.000000
75%,668.500000,1.000000,3.000000,37.000000,1.000000,0.000000,31.000000,0.000000,1.000000,1.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000


### Create a compute cluster
TODO: Describe the cluster here

In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cluster-1"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=10)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 2, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [4]:
# TODO: Put your automl settings here
automl_settings = {"max_concurrent_iterations": 9,
                   "primary_metric" : 'accuracy'
}
project_folder = './capstone-project'

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             experiment_timeout_minutes = 20,
                             training_data=dataset,
                             label_column_name="Survived",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [5]:
# TODO: Submit your experiment
# RemoteRun is an object of type AutoMLRun
remote_run = experiment.submit(automl_config, show_output=False)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
best_run_automl, fitted_model_automl = remote_run.get_output()
model_name = best_run_automl.properties['model_name']

In [10]:
print(best_run_automl)

Run(Experiment: titanic-automl-experiment,
Id: AutoML_387a10e5-a092-4507-b203-d4967d0fcf1e_46,
Type: azureml.scriptrun,
Status: Completed)


In [11]:
print(fitted_model_automl)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                             LogisticRegression(C=3237.45754281764,
                                                                                                class_weight='balanced',
                                                                                                dual=False,
                                               

In [12]:
best_run_automl.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'azureml-logs/55_azureml-execution-tvmps_1e96e7eb5b2ce3ca58434594aade5cbb6a7192a5ffb9e1143d85ae32a19013e7_d.txt',
 'azureml-logs/65_job_prep-tvmps_1e96e7eb5b2ce3ca58434594aade5cbb6a7192a5ffb9e1143d85ae32a19013e7_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_1e96e7eb5b2ce3ca58434594aade5cbb6a7192a5ffb9e1143d85ae32a19013e7_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'confusion_matrix',
 'explanation/77842347/classes.interpret.json',
 'explanation/77842347/eval_data_viz.interpret.json',
 'explanation/77842347/expected_values.interpret.json',
 'explanation/77842347/features.interpret.json',
 'explanation/77842347/global_names/0.interpret.json',
 'explanation/77842347/global_rank/0.interpret.json',
 'explanation/77842347/global_values/0.interpret.json',
 'explanation/77842347/local_importance_values.interpret.json',
 'explanation/77842347/per_class_names/0.interpret.json',
 'explanati

In [13]:
#TODO: Save the best model
model_name = 'best_model_titanic'
model = best_run_automl.register_model(model_name, model_path='outputs/model.pkl')

In [14]:
best_run_automl.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
best_run_automl.download_file('outputs/model.pkl', 'inference/model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [15]:
service_name = 'my-titanic-service'
inference_config = InferenceConfig(entry_script='inference/score.py')
service = Model.deploy(ws, service_name, [model], inference_config, overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.....................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [16]:
print(service.state)

Healthy



TODO: In the cell below, send a request to the web service you deployed to test it.

In [20]:
import json

data = {"data":
        [
          {
            "PassengerId": 812,
            "Pclass": 2,
            "Name": "Berriman, Mr. William John",
            "Age": 23.0,
            "SibSp": 0,
            "Parch": 0, 
            "Ticket": "28425",
            "Fare": 13.0,
            "Cabin": "",
            "Q": 0,
            "S": 1,
            "male": 1
          },
          {
            "PassengerId": 813,
            "Pclass": 1,
            "Name": "Lesurer, Mr. Gustave J",
            "Age": 35.0,
            "SibSp": 0,
            "Parch": 0, 
            "Ticket": "PC 17755",
            "Fare": 512.3292,
            "Cabin": "B101",
            "Q": 0,
            "S": 0,
            "male": 1
          }
      ]
    }

# Convert to JSON string
input_data = json.dumps(data)

TODO: In the cell below, print the logs of the web service and delete the service

In [21]:
output = service.run(input_data)

In [22]:
print(output)

{"result": [0.0, 1.0]}


In [23]:
service.delete()

In [25]:
print(service.state)

Deleting
